In [39]:
# Solar Irradiation Data Comparison - Thulhiriya
## Comparing CAMS, Solcast, and Actual Sensor Data with Kalman Filtering

In [40]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## 1. Kalman Filter Implementation
The Kalman filter will help smooth out noise in the solar irradiation measurements.

In [41]:
class KalmanFilter:
    """
    Simple Kalman Filter for 1D data smoothing
    """
    def __init__(self, process_variance=1e-5, measurement_variance=0.01, initial_value=0, initial_estimate_error=1):
        """
        Initialize Kalman Filter
        
        Parameters:
        - process_variance: How much we expect the true value to change (Q)
        - measurement_variance: Measurement noise variance (R)
        - initial_value: Initial state estimate
        - initial_estimate_error: Initial estimation error covariance (P)
        """
        self.process_variance = process_variance  # Q
        self.measurement_variance = measurement_variance  # R
        self.estimate = initial_value  # x
        self.estimate_error = initial_estimate_error  # P
        
    def update(self, measurement):
        """
        Update the Kalman filter with a new measurement
        
        Parameters:
        - measurement: New measurement value
        
        Returns:
        - Updated estimate
        """
        # Prediction step
        # Prior estimate = current estimate (assuming constant model)
        prior_estimate = self.estimate
        prior_estimate_error = self.estimate_error + self.process_variance
        
        # Update step
        # Calculate Kalman Gain
        kalman_gain = prior_estimate_error / (prior_estimate_error + self.measurement_variance)
        
        # Update estimate with measurement
        self.estimate = prior_estimate + kalman_gain * (measurement - prior_estimate)
        
        # Update estimate error
        self.estimate_error = (1 - kalman_gain) * prior_estimate_error
        
        return self.estimate
    
    def filter_data(self, data):
        """
        Apply Kalman filter to entire dataset
        
        Parameters:
        - data: Array-like data to filter
        
        Returns:
        - Filtered data array
        """
        filtered_data = []
        for measurement in data:
            if np.isnan(measurement):
                filtered_data.append(np.nan)
            else:
                filtered_value = self.update(measurement)
                filtered_data.append(filtered_value)
        return np.array(filtered_data)

## 2. Load CAMS Data (No Filtering)

In [42]:
# Load CAMS data
cams_file = 'CAMS_Thulhiriya_nov1_8_2025.csv'
cams_df = pd.read_csv(cams_file, sep=';', skiprows=0)

# The first column has '#' in the header, clean it
cams_df.columns = [col.strip().replace('# ', '') for col in cams_df.columns]

# Parse the observation period to get timestamp
# The format is like "2025-11-01T00:00:00.0/2025-11-01T00:01:00.0"
cams_df['timestamp'] = cams_df['Observation period'].apply(lambda x: x.split('/')[0])
cams_df['timestamp'] = pd.to_datetime(cams_df['timestamp'])

# Select relevant columns
cams_df = cams_df[['timestamp', 'GHI', 'DHI', 'BNI', 'BHI']].copy()

print("CAMS Data:")
print(f"Date range: {cams_df['timestamp'].min()} to {cams_df['timestamp'].max()}")
print(f"Total records: {len(cams_df)}")
print(f"\nFirst few rows:")
print(cams_df.head(10))
print(f"\nData statistics:")
print(cams_df[['GHI', 'DHI', 'BNI']].describe())

CAMS Data:
Date range: 2025-11-01 00:00:00 to 2025-11-08 23:59:00
Total records: 11520

First few rows:
            timestamp  GHI  DHI  BNI  BHI
0 2025-11-01 00:00:00  0.0  0.0  0.0  0.0
1 2025-11-01 00:01:00  0.0  0.0  0.0  0.0
2 2025-11-01 00:02:00  0.0  0.0  0.0  0.0
3 2025-11-01 00:03:00  0.0  0.0  0.0  0.0
4 2025-11-01 00:04:00  0.0  0.0  0.0  0.0
5 2025-11-01 00:05:00  0.0  0.0  0.0  0.0
6 2025-11-01 00:06:00  0.0  0.0  0.0  0.0
7 2025-11-01 00:07:00  0.0  0.0  0.0  0.0
8 2025-11-01 00:08:00  0.0  0.0  0.0  0.0
9 2025-11-01 00:09:00  0.0  0.0  0.0  0.0

Data statistics:
                GHI           DHI           BNI
count  11520.000000  11520.000000  11520.000000
mean       3.156745      1.888516      1.816720
std        4.558444      2.598557      3.472639
min        0.000000      0.000000      0.000000
25%        0.000000      0.000000      0.000000
50%        0.000000      0.000000      0.000000
75%        5.910375      3.395500      2.188325
max       16.129900      9.07560

## 3. Load Solcast Data (No Filtering)

In [43]:
# Load Solcast data
solcast_file = 'solcast_nov_2025_1stweek_minitly.csv'
solcast_df = pd.read_csv(solcast_file)

# Parse timestamp
solcast_df['timestamp'] = pd.to_datetime(solcast_df['period_end'])

# Select relevant columns
solcast_df = solcast_df[['timestamp', 'ghi', 'dhi', 'dni', 'gti']].copy()
solcast_df.rename(columns={'ghi': 'GHI', 'dhi': 'DHI', 'dni': 'DNI', 'gti': 'GTI'}, inplace=True)

print("Solcast Data:")
print(f"Date range: {solcast_df['timestamp'].min()} to {solcast_df['timestamp'].max()}")
print(f"Total records: {len(solcast_df)}")
print(f"\nFirst few rows:")
print(solcast_df.head(10))
print(f"\nData statistics:")
print(solcast_df[['GHI', 'DHI', 'DNI']].describe())

Solcast Data:
Date range: 2025-11-01 00:05:00+05:30 to 2025-11-08 00:00:00+05:30
Total records: 2016

First few rows:
                  timestamp  GHI  DHI  DNI  GTI
0 2025-11-01 00:05:00+05:30    0    0    0    0
1 2025-11-01 00:10:00+05:30    0    0    0    0
2 2025-11-01 00:15:00+05:30    0    0    0    0
3 2025-11-01 00:20:00+05:30    0    0    0    0
4 2025-11-01 00:25:00+05:30    0    0    0    0
5 2025-11-01 00:30:00+05:30    0    0    0    0
6 2025-11-01 00:35:00+05:30    0    0    0    0
7 2025-11-01 00:40:00+05:30    0    0    0    0
8 2025-11-01 00:45:00+05:30    0    0    0    0
9 2025-11-01 00:50:00+05:30    0    0    0    0

Data statistics:
               GHI          DHI          DNI
count  2016.000000  2016.000000  2016.000000
mean    194.327877   121.718254   106.405258
std     263.387126   166.963796   205.175161
min       0.000000     0.000000     0.000000
25%       0.000000     0.000000     0.000000
50%       0.000000     0.000000     0.000000
75%     406.250000   

## 4. Load Actual Sensor Data and Apply Kalman Filter
The actual sensor data (thulhirya) contains noise and will be smoothed using Kalman filtering.

In [44]:
# Load actual sensor data
sensor_file = 'thulhirya_nov_1stweek_minitly.csv'
sensor_df = pd.read_csv(sensor_file, sep=';')

# Parse timestamp
sensor_df['timestamp'] = pd.to_datetime(sensor_df['Timestamp'])
sensor_df = sensor_df[['timestamp', 'WS_rad']].copy()

print("Sensor Data (Before Kalman Filter):")
print(f"Date range: {sensor_df['timestamp'].min()} to {sensor_df['timestamp'].max()}")
print(f"Total records: {len(sensor_df)}")
print(f"\nFirst few rows:")
print(sensor_df.head(10))
print(f"\nData statistics:")
print(sensor_df['WS_rad'].describe())

# Apply Kalman Filter to sensor data
print("\n" + "="*60)
print("Applying Kalman Filter to Sensor Data...")
print("="*60)

kf_sensor = KalmanFilter(
    process_variance=1e-5,
    measurement_variance=0.1,
    initial_value=sensor_df['WS_rad'].iloc[0] if sensor_df['WS_rad'].iloc[0] > 0 else 0,
    initial_estimate_error=1
)
sensor_df['WS_rad_filtered'] = kf_sensor.filter_data(sensor_df['WS_rad'].values)

print("\nKalman filtering completed!")
print(f"Original data range: {sensor_df['WS_rad'].min():.2f} to {sensor_df['WS_rad'].max():.2f} W/m²")
print(f"Filtered data range: {sensor_df['WS_rad_filtered'].min():.2f} to {sensor_df['WS_rad_filtered'].max():.2f} W/m²")
print(f"\nSample comparison:")
print(sensor_df[['timestamp', 'WS_rad', 'WS_rad_filtered']].head(20))

Sensor Data (Before Kalman Filter):
Date range: 2025-11-01 00:00:30 to 2025-11-07 23:59:21
Total records: 9273

First few rows:
            timestamp  WS_rad
0 2025-11-01 00:00:30     0.0
1 2025-11-01 00:01:33     0.0
2 2025-11-01 00:02:36     0.0
3 2025-11-01 00:03:39     0.0
4 2025-11-01 00:04:42     0.0
5 2025-11-01 00:05:45     0.0
6 2025-11-01 00:06:48     0.0
7 2025-11-01 00:07:51     0.0
8 2025-11-01 00:08:54     0.0
9 2025-11-01 00:09:57     0.0

Data statistics:
count    9273.000000
mean      192.897876
std       298.682773
min         0.000000
25%         0.000000
50%         0.000000
75%       303.000000
max      1224.000000
Name: WS_rad, dtype: float64

Applying Kalman Filter to Sensor Data...

Kalman filtering completed!
Original data range: 0.00 to 1224.00 W/m²
Filtered data range: 0.00 to 840.47 W/m²

Sample comparison:
             timestamp  WS_rad  WS_rad_filtered
0  2025-11-01 00:00:30     0.0              0.0
1  2025-11-01 00:01:33     0.0              0.0
2  2025-1

## 5. Data Alignment and Preparation
Align all three datasets for comparison by merging on timestamp.

In [45]:
# Convert all timestamps to timezone-naive for easier merging
cams_df['timestamp'] = pd.to_datetime(cams_df['timestamp']).dt.tz_localize(None)
solcast_df['timestamp'] = pd.to_datetime(solcast_df['timestamp']).dt.tz_localize(None)
sensor_df['timestamp'] = pd.to_datetime(sensor_df['timestamp']).dt.tz_localize(None)

# Resample Solcast from 5-minute to 1-minute using interpolation
solcast_resampled = solcast_df.set_index('timestamp').resample('1T').interpolate(method='linear').reset_index()

# Merge all datasets
# First merge CAMS and Solcast
merged_df = pd.merge(
    cams_df,
    solcast_resampled,
    on='timestamp',
    how='outer',
    suffixes=('_cams', '_solcast')
)

# Then merge with sensor data
merged_df = pd.merge(
    merged_df,
    sensor_df,
    on='timestamp',
    how='outer'
)

# Sort by timestamp
merged_df = merged_df.sort_values('timestamp').reset_index(drop=True)

# Filter to common date range where all three datasets have data
common_start = max(cams_df['timestamp'].min(), solcast_df['timestamp'].min(), sensor_df['timestamp'].min())
common_end = min(cams_df['timestamp'].max(), solcast_df['timestamp'].max(), sensor_df['timestamp'].max())
merged_df = merged_df[(merged_df['timestamp'] >= common_start) & (merged_df['timestamp'] <= common_end)]

# Remove rows with all NaN values for the key columns
merged_df = merged_df.dropna(subset=['GHI_cams', 'GHI_solcast', 'WS_rad', 'WS_rad_filtered'], how='all')

print("Merged Dataset:")
print(f"Date range: {merged_df['timestamp'].min()} to {merged_df['timestamp'].max()}")
print(f"Total records: {len(merged_df)}")
print(f"\nSample data:")
print(merged_df[['timestamp', 'GHI_cams', 'GHI_solcast', 'WS_rad', 'WS_rad_filtered']].head(20))

Merged Dataset:
Date range: 2025-11-01 00:05:00 to 2025-11-07 23:59:21
Total records: 18912

Sample data:
             timestamp  GHI_cams  GHI_solcast  WS_rad  WS_rad_filtered
10 2025-11-01 00:05:00       0.0          0.0     NaN              NaN
11 2025-11-01 00:05:45       NaN          NaN     0.0              0.0
12 2025-11-01 00:06:00       0.0          0.0     NaN              NaN
13 2025-11-01 00:06:48       NaN          NaN     0.0              0.0
14 2025-11-01 00:07:00       0.0          0.0     NaN              NaN
15 2025-11-01 00:07:51       NaN          NaN     0.0              0.0
16 2025-11-01 00:08:00       0.0          0.0     NaN              NaN
17 2025-11-01 00:08:54       NaN          NaN     0.0              0.0
18 2025-11-01 00:09:00       0.0          0.0     NaN              NaN
19 2025-11-01 00:09:57       NaN          NaN     0.0              0.0
20 2025-11-01 00:10:00       0.0          0.0     NaN              NaN
21 2025-11-01 00:11:00       0.0          

## 6. Comparison Statistics
Calculate comparison metrics between the three datasets.

In [46]:
# Calculate statistics for comparison
print("="*80)
print("IRRADIATION DATA COMPARISON STATISTICS")
print("="*80)

print("\n--- CAMS Data ---")
print(f"Mean GHI: {merged_df['GHI_cams'].mean():.2f} W/m²")
print(f"Max GHI: {merged_df['GHI_cams'].max():.2f} W/m²")
print(f"Std Dev: {merged_df['GHI_cams'].std():.2f} W/m²")

print("\n--- Solcast Data ---")
print(f"Mean GHI: {merged_df['GHI_solcast'].mean():.2f} W/m²")
print(f"Max GHI: {merged_df['GHI_solcast'].max():.2f} W/m²")
print(f"Std Dev: {merged_df['GHI_solcast'].std():.2f} W/m²")

print("\n--- Sensor Data (Original) ---")
print(f"Mean Irradiation: {merged_df['WS_rad'].mean():.2f} W/m²")
print(f"Max Irradiation: {merged_df['WS_rad'].max():.2f} W/m²")
print(f"Std Dev: {merged_df['WS_rad'].std():.2f} W/m²")

print("\n--- Sensor Data (Kalman Filtered) ---")
print(f"Mean Irradiation: {merged_df['WS_rad_filtered'].mean():.2f} W/m²")
print(f"Max Irradiation: {merged_df['WS_rad_filtered'].max():.2f} W/m²")
print(f"Std Dev: {merged_df['WS_rad_filtered'].std():.2f} W/m²")

# Calculate differences
merged_df['diff_cams_sensor'] = merged_df['GHI_cams'] - merged_df['WS_rad_filtered']
merged_df['diff_solcast_sensor'] = merged_df['GHI_solcast'] - merged_df['WS_rad_filtered']
merged_df['diff_cams_solcast'] = merged_df['GHI_cams'] - merged_df['GHI_solcast']

print("\n" + "="*80)
print("DIFFERENCES (vs Filtered Sensor Data)")
print("="*80)
print(f"\nCAMS vs Sensor:")
print(f"  Mean Difference: {merged_df['diff_cams_sensor'].mean():.2f} W/m²")
print(f"  Mean Absolute Difference: {abs(merged_df['diff_cams_sensor']).mean():.2f} W/m²")
print(f"  RMSE: {np.sqrt((merged_df['diff_cams_sensor']**2).mean()):.2f} W/m²")

print(f"\nSolcast vs Sensor:")
print(f"  Mean Difference: {merged_df['diff_solcast_sensor'].mean():.2f} W/m²")
print(f"  Mean Absolute Difference: {abs(merged_df['diff_solcast_sensor']).mean():.2f} W/m²")
print(f"  RMSE: {np.sqrt((merged_df['diff_solcast_sensor']**2).mean()):.2f} W/m²")

print(f"\nCAMS vs Solcast:")
print(f"  Mean Difference: {merged_df['diff_cams_solcast'].mean():.2f} W/m²")
print(f"  Mean Absolute Difference: {abs(merged_df['diff_cams_solcast']).mean():.2f} W/m²")
print(f"  RMSE: {np.sqrt((merged_df['diff_cams_solcast']**2).mean()):.2f} W/m²")
print("="*80)

IRRADIATION DATA COMPARISON STATISTICS

--- CAMS Data ---
Mean GHI: 3.21 W/m²
Max GHI: 16.13 W/m²
Std Dev: 4.60 W/m²

--- Solcast Data ---
Mean GHI: 194.42 W/m²
Max GHI: 922.00 W/m²
Std Dev: 263.29 W/m²

--- Sensor Data (Original) ---
Mean Irradiation: 193.00 W/m²
Max Irradiation: 1224.00 W/m²
Std Dev: 298.73 W/m²

--- Sensor Data (Kalman Filtered) ---
Mean Irradiation: 192.97 W/m²
Max Irradiation: 840.47 W/m²
Std Dev: 233.59 W/m²

DIFFERENCES (vs Filtered Sensor Data)

CAMS vs Sensor:
  Mean Difference: -184.70 W/m²
  Mean Absolute Difference: 187.82 W/m²
  RMSE: 300.61 W/m²

Solcast vs Sensor:
  Mean Difference: -5.94 W/m²
  Mean Absolute Difference: 90.69 W/m²
  RMSE: 132.94 W/m²

CAMS vs Solcast:
  Mean Difference: -191.22 W/m²
  Mean Absolute Difference: 194.12 W/m²
  RMSE: 325.51 W/m²


## 7. Interactive Zoomable Comparison Plot
All datasets compared on a single interactive plot with zoom, pan, and hover capabilities.

In [47]:
# Create plot dataframe with continuous data (forward and backward fill)
plot_df = merged_df.copy()
plot_df['GHI_solcast'] = plot_df['GHI_solcast'].fillna(method='ffill').fillna(method='bfill')
plot_df['WS_rad_filtered'] = plot_df['WS_rad_filtered'].fillna(method='ffill').fillna(method='bfill')

# Create interactive zoomable Plotly plot
fig = go.Figure()

# Add Solcast data (orange line)
fig.add_trace(
    go.Scatter(
        x=plot_df['timestamp'], 
        y=plot_df['GHI_solcast'],
        mode='lines',
        name='Solcast GHI',
        line=dict(color='#FFA500', width=2),
        connectgaps=True,
        hovertemplate='<b>Solcast GHI</b><br>Time: %{x}<br>Irradiation: %{y:.2f} W/m²<extra></extra>'
    )
)

# Add filtered sensor data (brown line)
fig.add_trace(
    go.Scatter(
        x=plot_df['timestamp'], 
        y=plot_df['WS_rad_filtered'],
        mode='lines',
        name='Thulhiriya Sensor (Kalman Filtered)',
        line=dict(color='#8B4513', width=2.5),
        connectgaps=True,
        hovertemplate='<b>Thulhiriya Sensor (Filtered)</b><br>Time: %{x}<br>Irradiation: %{y:.2f} W/m²<extra></extra>'
    )
)

# Update layout
fig.update_layout(
    title={
        'text': '<b>Solar Irradiation Comparison - Solcast vs Thulhiriya Sensor (Nov 1-7, 2025)</b><br><i>Interactive Zoomable Plot - Click and drag to zoom, double-click to reset</i>',
        'font': {'size': 18}
    },
    xaxis_title='<b>Date and Time</b>',
    yaxis_title='<b>Solar Irradiation (W/m²)</b>',
    height=700,
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        font=dict(size=12)
    ),
    template='plotly_white',
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.5
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.5
    )
)

fig.show()
